In [ ]:
pip install pynrrd nibabel numpy

In [ ]:
import nrrd
import nibabel as nib
import numpy as np
import os

# Define input and output directories
input_folder = "/path/to/nrrd/"
output_folder = "/path/to/nifti/"

os.makedirs(output_folder, exist_ok=True)

for file in os.listdir(input_folder):
    if file.endswith(".nrrd"):
        file_path = os.path.join(input_folder, file)
        data, header = nrrd.read(file_path)  # Read NRRD file
        
        # Convert to NIfTI format
        nifti_img = nib.Nifti1Image(data, affine=np.eye(4))
        
        # Save the converted file
        output_path = os.path.join(output_folder, file.replace(".nrrd", ".nii.gz"))
        nib.save(nifti_img, output_path)
        print(f"Converted: {file} → {output_path}")


In [ ]:
pip install SimpleITK

In [ ]:
#Cropping 
import SimpleITK as sitk
import os

input_folder = "/path/to/nifti/"
output_folder = "/path/to/cropped_nifti/"
os.makedirs(output_folder, exist_ok=True)

for file in os.listdir(input_folder):
    if file.endswith(".nii.gz"):
        img_path = os.path.join(input_folder, file)
        
        # Read the image
        img = sitk.ReadImage(img_path)
        array = sitk.GetArrayFromImage(img)

        # Define crop range (adjust based on dataset)
        crop_slices = (50, 150)  # Adjust Z-axis range
        cropped_array = array[crop_slices[0]:crop_slices[1], :, :]
        
        # Convert back to NIfTI
        cropped_img = sitk.GetImageFromArray(cropped_array)
        cropped_img.SetSpacing(img.GetSpacing())
        cropped_img.SetDirection(img.GetDirection())
        cropped_img.SetOrigin(img.GetOrigin())

        # Save cropped image
        output_path = os.path.join(output_folder, file)
        sitk.WriteImage(cropped_img, output_path)
        print(f"Cropped and saved: {file}")


bash

pip install nnunet


bash

export nnUNet_raw_data_base="/path/to/nnUNet_raw_data_base"
export nnUNet_preprocessed="/path/to/nnUNet_preprocessed"
export nnUNet_results="/path/to/nnUNet_results"

# Create required folders
mkdir -p $nnUNet_raw_data_base/Task001_ThyroidSeg/imagesTr
mkdir -p $nnUNet_raw_data_base/Task001_ThyroidSeg/labelsTr
mkdir -p $nnUNet_raw_data_base/Task001_ThyroidSeg/imagesTs
✅ Store cropped images and labels in these folders.

#Preprocess Data for nnUNet

bash

nnUNetv2_plan_and_preprocess -d 1 --verify_dataset_integrity
✅ Output: Preprocessed images in nnUNet format.

#Train nnUNet Model

bash

nnUNetv2_train 3d_fullres Task001_ThyroidSeg 0

✅ Training starts using GPU.

#Evaluate Model Performance

bash

nnUNetv2_evaluate_predictions --gt_labels /path/to/labelsTs/ --predictions /path/to/predictions/

✅ Metrics (Dice Score, Hausdorff Distance) are computed.


#Inference on New Images

bash

nnUNetv2_predict -i /path/to/new_cts/ -o /path/to/predictions/ -d 1 -c 3d_fullres -f all
